<a href="https://colab.research.google.com/github/SnehalYu/Query-Auto-Completion-/blob/main/Sample_dateset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
#LM on prev dataset
import requests
import io


# Downloading the dataset
url = "https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt"
text = requests.get(url).content.decode('utf8')
with io.open('language-never-random.txt', 'w', encoding='utf8') as fout:
 fout.write(text)
characters = []
count=0
index=0
flag=0

# To count number of lines
for char in text:
    characters.append(char.lower())
    if(char=='\n'):
      count=count+1
    flag=flag+1
x=0.6*flag

char_unique=np.array(characters)
cu=np.unique(char_unique)

#Trainset creation
trainsetcount=0
trainset=[]
trainset.append('<s>')
trainset.append('<s>')
for i in text[:int(x)]:
  trainset.append(i.lower())
  trainsetcount=trainsetcount+1
trainset.append('</s>')
trainset.append('</s>')

#Dictionary Creation
dict_two={}
dict_three={}
for i in range(trainsetcount-1):
    pair_two = tuple(trainset[i:i+2])
    pair_three = tuple(trainset[i:i+3])
    if pair_two in dict_two:
        dict_two[pair_two] += 1
    else:
        dict_two[pair_two] = 1
    if pair_three in dict_three:
        dict_three[pair_three] += 1
    else:
        dict_three[pair_three] = 1

char_index={value: index for index, value in enumerate(cu)}
dict_bi_to_num={value: index for index, value in enumerate(dict_two)}
dict_num_to_bi={index: value for index, value in enumerate(dict_two)}


#Creating an array contaioning the trigram, bigram and probabilities
denom=0
arr=[]
size=[]
size.append(len(dict_two))
size.append(len(cu))
table=np.zeros(size)
for i in dict_two:
  for c in cu:
    arr=[]
    arr.append(i[0])
    arr.append(i[1])
    arr.append(c)
    pair=tuple(arr)
    if pair in dict_three:
      num = dict_three.get(pair)+1
      denom = dict_two.get(i)+len(cu)
      table[dict_bi_to_num.get(i)][char_index.get(c)]=num/denom
    else:
      num = 1
      denom = dict_two.get(i)+len(cu)
      table[dict_bi_to_num.get(i)][char_index.get(c)]=num/denom




In [10]:
print(dict_two)
print(dict_three)

{('<s>', '<s>'): 1, ('<s>', 'f'): 1, ('f', 'i'): 835, ('i', 'r'): 1580, ('r', 's'): 1792, ('s', 't'): 4248, ('t', ' '): 9774, (' ', 'c'): 3826, ('c', 'i'): 732, ('i', 't'): 3768, ('t', 'i'): 1999, ('i', 'z'): 279, ('z', 'e'): 219, ('e', 'n'): 5680, ('n', ':'): 416, (':', '\n'): 5140, ('\n', 'b'): 1110, ('b', 'e'): 2665, ('e', 'f'): 592, ('f', 'o'): 2392, ('o', 'r'): 5746, ('r', 'e'): 6241, ('e', ' '): 16809, (' ', 'w'): 6599, ('w', 'e'): 2105, (' ', 'p'): 2782, ('p', 'r'): 1098, ('r', 'o'): 2747, ('o', 'c'): 187, ('c', 'e'): 2348, ('e', 'e'): 2467, ('e', 'd'): 2347, ('d', ' '): 9102, (' ', 'a'): 7979, ('a', 'n'): 7645, ('n', 'y'): 266, ('y', ' '): 6759, (' ', 'f'): 4032, ('f', 'u'): 382, ('u', 'r'): 3347, ('r', 't'): 1494, ('t', 'h'): 16091, ('h', 'e'): 11584, ('e', 'r'): 7304, ('r', ','): 754, (',', ' '): 8275, (' ', 'h'): 7418, ('e', 'a'): 4024, ('a', 'r'): 5358, ('r', ' '): 6261, (' ', 'm'): 6825, ('m', 'e'): 4276, (' ', 's'): 7479, ('s', 'p'): 715, ('p', 'e'): 1269, ('a', 'k'): 957

In [25]:
print(table)

[[1.46854588e-07 1.46854588e-07 1.46854588e-07 ... 2.85720286e-03
  1.46854588e-07 2.93709175e-07]
 [8.98852974e-07 8.98852974e-07 2.63094265e-03 ... 8.26944736e-05
  8.98852974e-07 8.98852974e-07]
 [7.78433301e-04 9.90112531e-03 5.20717251e-01 ... 1.36567246e-05
  1.36567246e-05 1.36567246e-05]
 ...
 [1.08695652e-02 1.08695652e-02 1.41304348e-01 ... 1.08695652e-02
  1.08695652e-02 1.08695652e-02]
 [1.29870130e-02 1.29870130e-02 1.29870130e-02 ... 1.29870130e-02
  1.29870130e-02 1.29870130e-02]
 [2.08333333e-02 2.08333333e-02 2.08333333e-02 ... 2.08333333e-02
  2.08333333e-02 2.08333333e-02]]


In [24]:
#Sampling
def sample_next_character(context):
    bigram = tuple(context[-2:])
    if bigram in dict_two:
        probabilities = table[dict_bi_to_num[bigram]]
        next_char_index = np.random.multinomial(1, probabilities).argmax()
        next_char = cu[next_char_index]
    else:
        next_char = np.random.choice(cu)
    return next_char


def generate_text(starting_context, length):
    context = starting_context

    for _ in range(length):
        next_char = sample_next_character(context)
        context.append(next_char)

    generated_text = ''.join(context)
    return generated_text

starting_context = ['<s>', '<s>']  # Initial context to start text generation
length = 100  # Length of the generated text

generated_text = generate_text(starting_context, length)
print(generated_text)

<s><s>stra!lr!kv!qjame!bgrouty com govismask!schaterno!l,tmacescom on com cadsjoblast!f sped!nbal!a:g&pdai
